# The epidemic threshold of an ER network

In [105]:
import numpy
import networkx
from epyc import LabNotebook, ParallelLab, RepeatedExperiment
from epydemic import StochasticDynamics, SIR 

from jupyter_dash import JupyterDash
from dash.dependencies import Input, Output
import plotly.express as px
import plotly.graph_objects as go
import dash_core_components as dcc
import dash_html_components as html

In [2]:
nb = LabNotebook()
lab = ParallelLab(nb, cores=1)

In [3]:
N = 1000
kmean = 8
phi = kmean / N

Tcrit = 1 / kmean

In [79]:
g = networkx.fast_gnp_random_graph(N, phi)
m = SIR()
e = StochasticDynamics(m, g)

In [106]:
rs = nb.addResultSet('epydemic-threshold-6')
lab[SIR.P_INFECT] = list(numpy.linspace(0.0, 0.1, num=20)) + list(numpy.linspace(0.1, 0.4, num=150)) + list(numpy.linspace(0.4, 1.0, num=80))
lab[SIR.P_REMOVE] = 1.0
lab[SIR.P_INFECTED] = 0.005
lab.runExperiment(RepeatedExperiment(e, 10))
df = rs.dataframe()

In [189]:
app = JupyterDash('epidemic-threshold')

app.layout = html.Div([
    dcc.Graph(id='epidemic-threshold-plot'),
    html.Label([
        html.Span(id='epidemic-threshold-range'),
        dcc.RangeSlider(id='epidemic-threshold-Tcrit',
                        min=0.0, max=1.0, step=0.001,
                        marks={0.1 * i: '{v:.1f}'.format(v=0.1 * i) for i in range(11)},
                        value=[0.0, 1.0],
                        allowCross=False)
    ]),
])

@app.callback(
    [Output('epidemic-threshold-plot', 'figure'), Output('epidemic-threshold-range', 'children')],
    [Input("epidemic-threshold-Tcrit", "value")]
)
def plot_epidemic_threshold(Ts):
    min_t = min(df[SIR.P_INFECT], key=lambda x: abs(x - Ts[0]))
    max_t = min(df[SIR.P_INFECT], key=lambda x: abs(x - Ts[1]))
    pdf = df[(df[SIR.P_INFECT] >= min_t) & (df[SIR.P_INFECT] <= max_t)]
    
    mdf_x = pdf[SIR.P_INFECT].unique()
    mdf_y = [ pdf[pdf[SIR.P_INFECT] == pInfect][SIR.REMOVED].mean() for pInfect in mdf_x ]
    
    fig = go.Figure(layout=dict(title=dict(text=r'Epidemic size for ER network $\phi$')))
    fig.add_trace(go.Scatter(x=pdf[SIR.P_INFECT], y=pdf[SIR.REMOVED],
                             mode='markers', marker=dict(color='powderblue'),
                             name='Individual samples'))
    fig.add_trace(go.Scatter(x=mdf_x, y=mdf_y,
                             mode='lines', line = dict(color='blue'),
                             name='Mean epidemic size'))
    fig.add_trace(go.Scatter(x=[Tcrit, Tcrit], y=[0.0, pdf[SIR.REMOVED].max()],
                             mode='lines', line=go.scatter.Line(color='gray'),
                             showlegend=False))
    
    label = '[{l:.2f}, {h:.2f}]'.format(l=min_t, h=max_t)
    
    return (fig, label)
            
app.run_server(mode='inline')